# Get data from Open Street Map API:
Input: Origin and destination coordinates (excel file)
Output: routes by car (as polylines = list of coordinates; distance; duration)
Export to csv

In [1]:
import requests
import json
import pandas as pd

In [2]:
#single call, examle

# lat_1, lon_1  =  45.650200,	 13.767620  # Trieste Piazza Unità
# lat_2, lon_2  =  45.656938,  13.829219   # campus Padriciano
# r = requests.get(f"http://router.project-osrm.org/route/v1/car/{lon_1},{lat_1};{lon_2},{lat_2}?overview=full&geometries=geojson""")
# routes = json.loads(r.content)
# route_1 = routes.get("routes")[0]


In [4]:
filename = r'orig-dest.xlsx'
orig_cols = ['n', 'OR', 'orig', 'Lat', 'Long']
dforig = pd.read_excel(filename, sheet_name = "ORIG", usecols = orig_cols)
dforig.set_index('n', inplace = True)
dforig.dropna(axis = 0, how = 'any', subset = ['Lat'], inplace = True)

dest_cols = ['n', 'DD', 'dest', 'Lat', 'Long']
dfdest = pd.read_excel(filename, sheet_name = "DEST", usecols = dest_cols)
dfdest.set_index('n', inplace = True)
dfdest.dropna(axis = 0, how = 'any', subset = ['Lat'], inplace = True)


In [6]:
all_routes = pd.DataFrame(columns= ['o', 'd', 'dist', 'dur', 'geom']) #empty

for i, orig in dforig.iterrows():
    lat_1, long_1  =  orig['Lat'], orig['Long'] 
    siglaO = orig["OR"]

    for j, dest in dfdest.iterrows():
        lat_2, long_2  =  dest['Lat'], dest['Long'] 
        siglaD = dest["DD"]
        
        r = requests.get(f"http://router.project-osrm.org/route/v1/car/{long_1},{lat_1};{long_2},{lat_2}?overview=full&geometries=geojson""")
        routes = json.loads(r.content)
        route_1 = routes.get("routes")[0] #select only forst route for each jorney
        dist =  int(route_1.get("distance")/10)/100
        dur =   route_1.get("duration") #seconds
        geom =  route_1.get("geometry")['coordinates']

        
        all_routes = all_routes.append({'o': siglaO, 'd': siglaD, 
                                        'dist': dist,
                                        'dur':dur, 
                                        'geom':geom}, ignore_index=True)
        
        # print(f'Origine {i} {siglaO} Lat {lat_1} Long {long_1}')
        # print(f'Destinazione {j} {siglaD} Lat {lat_2} Long {long_2}')
        # print(dist, dur)
        # print('\n\n')
        

Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 1 TS_P Lat 45.656938 Long 13.829219
10.56 747



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 2 TS_B Lat 45.644505 Long 13.846497
10.14 814.9



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 3 UD_FI Lat 46.014887 Long 13.267489
71.04 3225.1



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 4 UD_C Lat 46.063948 Long 13.231134
73.27 3299.5



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 5 UD_S Lat 46.085519 Long 13.206124
76.67 3435.6



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 6 UD_A Lat 46.370194 Long 13.077139
112.84 4551.6



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 7 LEF Lat 45.944881 Long 12.88089
91.31 5101.8



Origine 1 TS1 Lat 45.6502 Long 13.76762
Destinazione 8 PN Lat 45.932598 Long 12.67542
111.88 4581.7



Origine 2 TS2 Lat 45.649826 Long 13.777175
Destinazione 1 TS_P Lat 45.656938 Long 13.829219
9.89 727.4



Origine 2 TS2 Lat 45.649826 Long 13.777175
Destinazione 2 TS

In [7]:
all_routes.head(5)


,o,d,dist,dur,geom
0,TS1,TS_P,10.56,747,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
1,TS1,TS_B,10.14,814.9,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
2,TS1,UD_FI,71.04,3225.1,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
3,TS1,UD_C,73.27,3299.5,"[[13.767103, 45.650613], [13.76746, 45.650831]..."
4,TS1,UD_S,76.67,3435.6,"[[13.767103, 45.650613], [13.76746, 45.650831]..."


In [15]:
all_routes.to_csv('all_routes.csv', index = False)
